# Алгоритм автоматического поиска ошибок в транскрибациях 
## на основе марковских цепей и векторной модели Word2Vec

### Авторы:

#### Смирнова Екатерина,
#### Черная Анастасия 

## Загрузка библиотек

In [1]:
import pandas as pd
import re
import os
import logging
import opencorpora 
import pickle
import nltk
import gensim
import wget

from unicodedata import normalize

from nltk import word_tokenize

from string import punctuation

import pymorphy2
morph = pymorphy2.MorphAnalyzer() #сразу же инициализируем морфанализатор

from num2words import num2words
from fuzzywuzzy import fuzz

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from gensim.models import Word2Vec

unable to import 'smart_open.gcs', disabling that module


## Загрузка данных
### Корпуса для обучения языковой модели на основе марковских цепей

Состав объединенного корпуса: открытый корпус русского языка проекта OpenCorporа + корпус субтитров к фильмам и сериалам + корпус сказок для детей и взрослых

#### Корпус opencorpora

Загружаем тренировочный корпус - открытый корпус русского языка проекта [OpenCorpora](http://opencorpora.org/)  

Ссылка на корпус находится [здесь](http://opencorpora.org/?page=downloads): 

In [2]:
train_corpus = opencorpora.load('annot.opcorpora.xml')

In [3]:
#Количество документов и токенов:
train_corpus

<Corpus revision=4566500 docs:4030 tokens:1989618>

In [4]:
#Вытаскиваем токены и текст из них (по атрибуту source). Для этого применяем source из пакета opencorpora-tools:
#Лучше обойтись без токенизации от nltk, а взять токены из корпуса с помощью метода tokens - здесь чище токенизация
train_corp = [elem.source for elem in train_corpus.tokens]

In [5]:
train_corp[:5]

['«', 'Школа', 'злословия', '»', 'учит']

In [6]:
print('Количество токенов в тренировочном корпусе opencorpora до предобработки: ', len(train_corp))

Количество токенов в тренировочном корпусе opencorpora до предобработки:  1989618


#### Корпус субтитров к фильмам и сериалам

[Ссылка](https://github.com/Desklop/Russian_subtitles_dataset/tree/master/data) на загрузку файла

In [7]:
subtitles = open('subtitles_ru.txt', mode = 'r', encoding = "utf-8").read()

In [8]:
subtitles[:50]

'привет, пап.\nдоброе утро, девочки.\nжаль, меня не б'

#### Корпус сказок для детей и взрослых

[Ссылка](https://www.kaggle.com/idoldev/adult-and-child-russian-tales-dataset-with-label) на загрузку файла. 

In [9]:
tales_df = pd.read_csv('tales.csv')

In [10]:
tales_df.head()

,Tale,Label
0,"Снежно белый снеговик Он не мал и не велик, Сн...",1
1,Снеговик из детской сказки Раскраснелась детво...,1
2,Снежная баба Мы снежную бабу слепили на славу....,1
3,"Ком, комочек и комок Что стоишь и ждёшь, дружо...",1
4,Лепим снеговика Лепит с самого утра Детвора сн...,1


In [11]:
tales = list(tales_df.Tale.values)
tales = ' '.join([str(sent) for sent in tales])

### Корпус транскрибаций

[Ссылка](https://github.com/smekur/Spoken_Corpora_with_Kaldi/tree/master/Транскрибации/all_transcripts.xlsx) на загрузку корпуса с транскрибациями 

In [12]:
trans = pd.read_excel('all_transcripts.xlsx')

In [13]:
trans.head()

,audio_ID,alphacep_transcripts,abk_transcripts
0,NDS_001-f-z,NaN,азат аллё а потом вот опять заново уже а по по...
1,NDS_002-f-z,слуга прошла подальше от прошла галлия а этот ...,хорошо да да гулять а я я ульянова задачка у н...
2,NDS_003-f-z,NaN,например спасибо я назад одна с нами наименее ...
3,NDS_004-m-z,NaN,я я готова я не пошёл
4,NDS_005-m-z,NaN,говорите а да манакова


Т.к. транскрибации подкорпуса NDS получились очень плохого качества из-за специфики детской речи, в рамках данного проекта рассмотрим только подкорпуса Pic и Sib. 

Отфильтруем из общего датафрейма нужные нам транскрибации и разберем их по двум датафреймам в заивисмости от модели транскрибатора

In [14]:
trans_alpha = trans.loc[
       (trans.audio_ID.str.contains('Sib')|trans.audio_ID.str.contains('Pic'))&trans.alphacep_transcripts.notnull(),
       ['audio_ID','alphacep_transcripts']
    ]
trans_abk = trans.loc[
       (trans.audio_ID.str.contains('Sib')|trans.audio_ID.str.contains('Pic'))&trans.abk_transcripts.notnull(),
       ['audio_ID','abk_transcripts']
    ]

In [15]:
trans_alpha.head()

,audio_ID,alphacep_transcripts
35,Pic-RUS_01-f_Pr-R.zip,жилбыл один дяденька по его жены скоро должно ...
38,Pic-RUS_01-f_Ski-T.zip,генин жизни одного очень увлекающийся спортом ...
39,Pic-RUS_02-f_Pr-R.zip,однозначным был день рождения мышь решил подар...
42,Pic-RUS_02-f_Ski-T.zip,этот человек встал рано утром позавтракал а по...
45,Pic-RUS_03-m_Ski-R.zip,знакомым мне здесь рассказали одну смешную и п...


In [16]:
trans_abk.head()

,audio_ID,abk_transcripts
35,Pic-RUS_01-f_Pr-R.zip,жил был один дяденька его жены скоро должно бы...
36,Pic-RUS_01-f_Pr-T.zip,товарищ тут с ним случилось проблема у его жен...
37,Pic-RUS_01-f_Ski-R.zip,жил был один молодой человек это молодой челов...
38,Pic-RUS_01-f_Ski-T.zip,день жизнь одного точне увлекающейся спорту му...
39,Pic-RUS_02-f_Pr-R.zip,однозначно был день рождения её муж решил пода...


## Предобработка тренировочных корпусов, токенизация, лемматизация и статистика

Функция для обработки цифр: превращаем цифры в слова.

На вход подаем список

In [17]:
#Используем готовую функцию num2words из библиотеки num2words:
def convert_num2words (tokens_list):
    for i in range(len(tokens_list)):
        if re.fullmatch(r'\d+', tokens_list[i]) is not None:
            tokens_list[i] = num2words(tokens_list[i], lang='ru')
    new_tokens_list =' '.join(tokens_list).split() #чтобы сложные числительные разбить по словам и сохранить порядок следования токенов 
    return new_tokens_list

Функция для предобработки:
- убирает токены-пунктуатры
- приводит к нижему регистру
- заменяет 'ё' на 'е'

На вход подаем список токенов

In [18]:
def preprocess_tokens(tokens_list):
    fine_tokens = [re.sub(r'ё', r'е', elem.lower()) for elem in tokens_list if re.fullmatch(r'(?:.*[А-Яа-яЁё]+.*)', elem) is not None]
    return fine_tokens

Функцию, которая лемматизирует словоформы-токены с помощью морфанализатора pymorphy2

На вход подаем список токенов-словоформ

In [19]:
def morph_analyzator(tokens):
    morph_analized = [morph.normal_forms(token)[0] for token in tokens]
    return morph_analized

#### Предобработка корпуса opencorpora

In [20]:
#Конвертируем цифры в слова:
train_corp = convert_num2words(train_corp)

In [21]:
#Делаем базовую предобработку (подробно см. в описании функции):
preprocessed_train_corp = preprocess_tokens(train_corp)

In [22]:
preprocessed_train_corp[:10]

['школа',
 'злословия',
 'учит',
 'прикусить',
 'язык',
 'сохранится',
 'ли',
 'градус',
 'дискуссии',
 'в']

In [23]:
#Получаем словарь уникальных словоформ:
vocab_opencorp = set(preprocessed_train_corp)

In [24]:
#Лемматизируем корпус:
lemmas_opencorpora = morph_analyzator(preprocessed_train_corp)

In [25]:
print("Общий объем корпуса opencorpora после предобработки: " +str(len(preprocessed_train_corp)))

Общий объем корпуса opencorpora после предобработки: 1612160


In [26]:
print("Количество уникальных токенов в корпусе opencorpora после предобработки: " +str(len(set(preprocessed_train_corp))))

Количество уникальных токенов в корпусе opencorpora после предобработки: 163022


In [27]:
print("Количество уникальных лемм в корпусе opencorpora: ", str(len(set(lemmas_opencorpora))))

Количество уникальных лемм в корпусе opencorpora:  65335


#### Корпус субтитров

Сначала токенизируем с помощью nltk.

In [28]:
#Токенизируем с помощью nltk.word_tokenize:
subtitles_tokens = word_tokenize(subtitles)
len(subtitles_tokens)

36367392

In [29]:
subtitles_corp = convert_num2words(subtitles_tokens)

In [30]:
preprocessed_subtitles = preprocess_tokens(subtitles_corp)

In [31]:
preprocessed_subtitles[:10]

['привет',
 'пап',
 'доброе',
 'утро',
 'девочки',
 'жаль',
 'меня',
 'не',
 'было',
 'этим']

In [32]:
vocab_subtitles = set(preprocessed_subtitles)

In [33]:
%time
lemmas_subtitles = morph_analyzator(preprocessed_subtitles)

CPU times: user 17 µs, sys: 3 µs, total: 20 µs
Wall time: 3.99 ms


In [34]:
print("Общий объем корпуса субтитров после предобработки: " +str(len(preprocessed_subtitles)))

Общий объем корпуса субтитров после предобработки: 28241077


In [35]:
print("Количество уникальных токенов в корпусе субтитров после предобработки: " +str(len(set(preprocessed_subtitles))))

Количество уникальных токенов в корпусе субтитров после предобработки: 482251


In [36]:
print("Количество уникальных лемм в корпусе субтитров: ", str(len(set(lemmas_subtitles))))

Количество уникальных лемм в корпусе субтитров:  230166


#### Корпус сказок

In [37]:
#Прежде избавляемся от \xa0 (в сказках есть no-brake space): 
tales = normalize('NFKD', tales)

In [38]:
tales[:100]

'Снежно белый снеговик Он не мал и не велик, Снежно белый снеговик. У него морковкой нос, Очень лю'

In [39]:
len(tales)

12241202

In [40]:
tales_tokens = word_tokenize(tales)
len(tales_tokens)

2387363

In [41]:
tales_tokens[:10]

['Снежно', 'белый', 'снеговик', 'Он', 'не', 'мал', 'и', 'не', 'велик', ',']

In [42]:
tales_corp = convert_num2words(tales_tokens)

In [43]:
len(tales_corp)

2393444

In [44]:
preprocessed_tales = preprocess_tokens(tales_corp)

In [45]:
vocab_tales = set(preprocessed_tales)

In [46]:
lemmas_tales = morph_analyzator(preprocessed_tales)

In [47]:
print("Общий объем корпуса сказок после предобработки: " +str(len(preprocessed_tales)))

Общий объем корпуса сказок после предобработки: 1894231


In [48]:
print("Количество уникальных токенов в корпусе сказок после предобработки: " +str(len(set(preprocessed_tales))))

Количество уникальных токенов в корпусе сказок после предобработки: 149651


In [49]:
print("Количество уникальных лемм в корпусе субтитров: ", str(len(set(lemmas_tales))))

Количество уникальных лемм в корпусе субтитров:  68014


## Объединение обучающих корпусов

In [50]:
lemmas_all = lemmas_opencorpora+lemmas_subtitles+lemmas_tales

In [51]:
lemmas_all[:100]

['школа',
 'злословие',
 'учить',
 'прикусить',
 'язык',
 'сохраниться',
 'ли',
 'градус',
 'дискуссия',
 'в',
 'новый',
 'сезон',
 'великолепный',
 'школа',
 'злословие',
 'вернуться',
 'в',
 'эфир',
 'после',
 'летний',
 'каникулы',
 'в',
 'новый',
 'формат',
 'в',
 'история',
 'программа',
 'это',
 'уже',
 'не',
 'один',
 'ребрендинга',
 'сейчас',
 'с',
 'труд',
 'можно',
 'припомнить',
 'что',
 'начинаться',
 'школа',
 'на',
 'канал',
 'культура',
 'как',
 'стандартный',
 'ток-шоу',
 'который',
 'отличаться',
 'от',
 'другой',
 'кухонный',
 'обсуждение',
 'гость',
 'что',
 'называться',
 'за',
 'глаз',
 'и',
 'неожиданный',
 'персона',
 'в',
 'качество',
 'ведущий',
 'писательница',
 'татьяна',
 'толстой',
 'и',
 'сценаристка',
 'дуня',
 'смирнов',
 'вроде',
 'бы',
 'не',
 'вполне',
 'соответствовать',
 'принятый',
 'на',
 'российский',
 'телевидение',
 'стандарт',
 'телеведущая',
 'впрочем',
 'на',
 'канал',
 'культура',
 'в',
 'роль',
 'телеведущая',
 'выступать',
 'и',
 'писател

In [52]:
print("Количество лемм в объединенном корпусе: ", str(len(lemmas_all)))

Количество лемм в объединенном корпусе:  31747468


In [53]:
print("Количество уникальных лемм в объединенном корпусе: ", str(len(set(lemmas_all))))

Количество уникальных лемм в объединенном корпусе:  286016


In [54]:
# сохраняем лемматизированный корпус в файл: 
with open('lemmas_train_corp.txt', 'w', encoding='utf-8') as file:
    file = file.write(' '.join(lemmas_all))

#### Создание объединенного словаря словоформ

In [55]:
vocab = vocab_opencorp.union(vocab_subtitles, vocab_tales)

In [56]:
len(vocab)

592898

In [57]:
# сохраняем объединенный словарь словоформ в файл: 
with open('wordforms_train_corp.txt', 'w', encoding='utf-8') as file:
    file = file.write(' '.join(vocab))

### Список вводных слов и частотных дискурсивных маркеров

[Ссылка](https://github.com/smekur/Spoken_Corpora_with_Kaldi/tree/master/mistakes_search/dictionaries_txt) для скачивания 

Данный список создан самостоятельно на основе [списка]('http://new.gramota.ru/spravka/punctum/punctum-alphabet') из источника Грамота.ру, 
а также анализа частотных дискурсивных слов в золотом стандарте для наших транскрибаций

In [58]:
with open('parenthesis_rus.txt', encoding='utf-8') as file:
    parenthesis_words = file.read().split('\n')

In [59]:
parenthesis_words[:10]

['безусловно',
 'бесспорно',
 'бывает',
 'бывало',
 'вернее',
 'верно',
 'вероятно',
 'видать',
 'видимо',
 'видно']

### Список частотных слов с дефисом

[Ссылка](https://github.com/smekur/Spoken_Corpora_with_Kaldi/tree/master/mistakes_search/dictionaries_txt) для скачивания

Список создан самостоятельно на основе слов с дефисом из обучающего корпуса и словаря частотных лемм Шарова и Ляшевской

In [60]:
with open('words_with_hyphen.txt', encoding='utf-8') as file:
    hyphened_words = file.read().split('\n')

## Марковская цепь

#### Подсчет условных частот

In [61]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(lemmas_all))#для лемматизированного объединенного корпуса 

In [62]:
cfreq['кататься']

FreqDist({'на': 406, 'по': 63, 'с': 30, 'в': 28, 'я': 11, 'и': 9, 'он': 8, 'не': 8, 'вместе': 7, 'ты': 6, ...})

Сохраняем словарь модели c помощью pickle

In [63]:
save_cfreq = open("cfreq_nastya.pickle" , "wb" )
pickle.dump(cfreq, save_cfreq)
save_cfreq.close()

#### Подсчет условных вероятностей

In [64]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)#для лемматизированного объединенного корпуса

In [65]:
cprob['он'].prob('свалиться')

5.338916314393528e-05

Сохраняем словарь условных вероятностей модели c помощью pickle

In [127]:
save_cprob = open("cprob_nastya.pickle" , "wb" )
pickle.dump(cprob, save_cprob)
save_cprob.close()

## Объявление функций для предобработки тестового корпуса и нахождения ошибок текста

In [66]:
'''
Функция для предобработки текста транскрибации: замена ё/е, составление биграм
Аргументы:
    text - исследуемый текст
    parenthesis - если True, то мы исключаем из текста транскрибации вводные и частотные дискурсивные маркеры
Вывод:
    список биграм текста 
'''

def text_preprocess(text, parenthesis = True):
    text = text.replace('ё','е')
    if parenthesis ==True:
        splt_text = text.split() #разобьем на слова текст и проверим, если слово является вводным или дискурсивным маркером, мы его исключаем из обработки
        for word in splt_text.copy():
            if word in parenthesis_words: #список вводных слов и дискурсивных маркеров 
                splt_text.remove(word)
        text = ' '.join(splt_text) #собираем слова текста в одну строку
        
    bigram_vectorizer = CountVectorizer(ngram_range=(2,2),
                                    token_pattern=r'[а-я-]+', min_df=1)
    analyze = bigram_vectorizer.build_analyzer()
    bigrams = analyze(text) 
    return bigrams

In [67]:
'''
Функция для предобработки текста транскрибации: вставка дефиса для случаев "когдато"
Аргументы:
    word - исследуемое слово
    words_with_hyphen - справочник слов с дефисом
Вывод:
    слово с дефисом 
'''

def insert_hyphen(word, words_with_hyphen):
    splits = [(word[:i], word[i:]) for i in range (len(word) - 1)]
    for L, R in splits:
        if L + '-' + R in words_with_hyphen:
            word = L + '-' + R
    return word

In [68]:
'''
Функция для предобработки текста транскрибации: замена случаев "когдато" на слова с дефисом
Аргументы:
    text - исследуемый текст
    words_with_hyphen - справочник слов с дефисом
Вывод:
    скорректированный текст 
'''

def insert_hyphen_in_text(text, words_with_hyphen):
    correct_text = ''
    correct_words = []
    text = text.split()
    for word in text:
        correct_word = insert_hyphen(word, words_with_hyphen)
        if correct_word != word:
            correct_words.append(correct_word)
        correct_text = correct_text + correct_word + ' '
    return correct_text.strip()

In [69]:
'''
Функция для нахождения ошибок транскрибаций

Параметры:
    text - исходный текст для анализа (str)
    model_freq - словарь частот совместной встречаемости, полученный на основе обучающего корпуса (nltk.probability.ConditionalFreqDist)
    model_prob - словарь вероятности совместной встречаемости, полученный на основе обучающего корпуса (nltk.probability.ConditionalProbDist)
    lemma - лемматизация слов в биграмах - по умолчанию True (bool)
    threshold - порог для установления "ошибки" транскрибации - по умолчанию 0 (int)

Вывод:
    mistakes - список биграмм, в которых есть ошибка транскрибации (list)
    bigrams_prob_dict - словарь, содержащий биграммы, в которых нет ошибки, и вероятности совместной встречаемости слов биграмм не ниже порога (dict)
    no_such_word - список биграмм, в которых хотя бы одно слово отсутствует в словаре обучающего корпуса (list)

'''


def find_mistakes(text, model_freq, model_prob, lemma = True, threshold = 0):
    mistakes = [] #переменная для хранения "ошибок"
    no_such_word = [] #переменная для хранения биграм, в которых хотя бы одно слово отсутствует в обучающем корпусе
    bigrams_prob_dict = {} #переменная для хранения "правильных" биграм и их вероятностей
    
    TEXT = text_preprocess(text) #список биграм текста
    
    for bigram in TEXT:
        big = bigram.split()
        if lemma == True:
            big = [morph.normal_forms(token)[0] for token in big] #лемматизируем слова в биграмах
    
        if model_freq[big[0]]=={} or model_freq[big[1]]=={}: #проверяем, есть ли слово в нашем тренировочном корпусе
            no_such_word.append(bigram)
            continue
    
        else:
            #считаем ошибкой, если вероятность совместной встречаемости меньше или равна установленному порогу
            if model_prob[big[0]].prob(big[1]) <= threshold: 
                mistakes.append(bigram)
            else:
                bigrams_prob_dict[bigram] = model_prob[big[0]].prob(big[1])
                
    print('\tВсего ошибок: ', len(mistakes), ' из ', len(TEXT))
    print('\tКоличество биграм с отсутствующим в словаре словом: ', len(no_such_word))
    
    return mistakes, bigrams_prob_dict, no_such_word 

In [70]:
'''
Функция для нахождения слов-ошибок в биграммах - кандидатах на ошибку

Аргументы:
    mistakes - список биграмм-кандидатов на ошибку из рассматриваемого
    bigrams_prob_dict - словарь с "правильными" биграмами того же текста
    
Вывод:
    словарь вида: {`слово-ошибка`: `биграма-кандидат`}
'''

def find_true_mistakes(mistakes, bigrams_prob_dict):
    true_mistakes = {}
    for mist in mistakes:
        for word in mist.split():
            bigs = []
            for key in bigrams_prob_dict:
                bigs += key.split()
            if word in bigs:
                continue
            else:
                if word not in true_mistakes:
                    true_mistakes[word]= mist
    return true_mistakes

In [71]:
'''
Функция для нахождения слов в словаре тренировочного корпуса, схожих исходному по расстоянию Левенштейна

Аргументы:
    voc - словарь тренировочного корпуса (слова не лемматизированы!)
    true_mistakes - словарь слов-ошибок и их биграм

Вывод:
    словарь вида {`слово-ошибка`: [`список схожих слов`]}
'''


def find_variation(voc, true_mistakes):
   
    variates = {}
    for mistake in true_mistakes:
        words = []
        for word in voc:
            
            if fuzz.ratio(mistake, word) > 80: 
                words.append(word)
        variates[mistake] = words
    
    return variates

In [72]:
'''
Функция для поиска ошибок 2-х категорий в тексте транскрибации:
1-я категория - вероятность схожего слова, найденного по расстоянию Левенштейна, в данной биграме выше,чем вероятность исходного слова;
2-я категория - вероятность схожего слова та же или ниже (то есть замена не дала результатов);

Аргументы:
    leven_mist - словарь, где ключ - исходное слово-ошибка, значение - список схожих слов по Левенштейну
    true_mistakes - словарь слов-ошибок и их биграм
    cprob_all - модель для подсчета условной вероятности слов в биграме

Вывод:
    first_mistakes - список слов первой категории
    second_mistakes - список слов второй категории
    
'''


def categorize_mistakes(leven_mist, true_mistakes, cprob_all):
    
    first_mistakes = []
    second_mistakes = []
    new_words = []
    
    for key in leven_mist:
        candidates = [morph.normal_forms(cand)[0] for cand in leven_mist[key]]
                      
        pair = ''.join([word for word in true_mistakes[key].split() if word != key])
        pair_lem = morph.normal_forms(pair)[0]
        key_lem = morph.normal_forms(key)[0]
        
        for cand in candidates:
            if cand != key_lem:
                if cprob_all[cand].prob(pair_lem)> 0:
                    first_mistakes.append(key)
                    new_words.append(cand)
                    break
    
        second_mistakes.append(key)
    return first_mistakes, second_mistakes, new_words

## Работа с транскрибациями (только марковская цепь)

#### Тестирование модели с марковской цепью, **с учетом вводных слов и дискурсивных маркеров в транскрибациях**

#### Корпус транскрибатора Alphacep

In [73]:
%time
trans_alpha['mistakes'] = ''
trans_alpha['bigram_mist'] = ''
trans_alpha['new_words'] = ''
trans_alpha['mistakes_1st_type'] = ''
trans_alpha['mistakes_2nd_type'] = ''
trans_alpha['absent_words'] = ''


for row in trans_alpha.alphacep_transcripts.index:
    print('Обрабатываю ряд ', row)
    text = insert_hyphen_in_text(trans_alpha.alphacep_transcripts[row], hyphened_words)
    mistakes, bigrams_prob_dict, no_such_word = find_mistakes(text, cfreq, cprob)
    true_mistakes = find_true_mistakes(mistakes, bigrams_prob_dict)
    leven_mist = find_variation(vocab, true_mistakes)
    
    first_mistakes, second_mistakes, new_words = categorize_mistakes(leven_mist, true_mistakes, cprob)
    
    trans_alpha['mistakes'][row] = ', '.join([key for key in true_mistakes])
    trans_alpha['bigram_mist'][row] = ', '.join([true_mistakes[key] for key in true_mistakes])
    trans_alpha['new_words'][row] =', '.join(new_words)
    trans_alpha['absent_words'][row] = ', '.join(no_such_word)
    trans_alpha['mistakes_1st_type'][row] = ', '.join(first_mistakes)
    trans_alpha['mistakes_2nd_type'][row] = ', '.join(second_mistakes)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs
Обрабатываю ряд  35
	Всего ошибок:  24  из  137
	Количество биграм с отсутствующим в словаре словом:  2
Обрабатываю ряд  38
	Всего ошибок:  15  из  81
	Количество биграм с отсутствующим в словаре словом:  3
Обрабатываю ряд  39
	Всего ошибок:  12  из  78
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  42
	Всего ошибок:  5  из  65
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  45
	Всего ошибок:  19  из  83
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  46
	Всего ошибок:  16  из  68
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  52
	Всего ошибок:  31  из  112
	Количество биграм с отсутствующим в словаре словом:  2
Обрабатываю ряд  54
	Всего ошибок:  15  из  106
	Количество биграм с отсутствующим в словаре словом:  2
Обрабатываю ряд  55
	Всего ошибок:  5  из  39
	Количество биграм с отсутствующим в словаре словом:  0
Обра

In [74]:
trans_alpha.head()

,audio_ID,alphacep_transcripts,mistakes,bigram_mist,new_words,mistakes_1st_type,mistakes_2nd_type,absent_words
35,Pic-RUS_01-f_Pr-R.zip,жилбыл один дяденька по его жены скоро должно ...,"дяденька, плот, ночного, посол, салон, наставь...","один дяденька, случится плот, дяденька ночного...","дядька, ставить","дяденька, наставь","дяденька, плот, ночного, посол, салон, наставь...","посол варт, варт салон"
38,Pic-RUS_01-f_Ski-T.zip,генин жизни одного очень увлекающийся спортом ...,"хочется, партийный, товарищ, тоесть, природу, ...","катался хочется, хочется партийный, партийный ...",хотеть,хочется,"хочется, партийный, товарищ, тоесть, природу, ...","генин жизни, и норг, норг тоесть"
39,Pic-RUS_02-f_Pr-R.zip,однозначным был день рождения мышь решил подар...,"однозначным, мышь, посылала","однозначным был, рождения мышь, в посылала","однозначно, подослать","однозначным, посылала","однозначным, мышь, посылала",
42,Pic-RUS_02-f_Ski-T.zip,этот человек встал рано утром позавтракал а по...,"наложены, божественного","отправился наложены, голову божественного",,,"наложены, божественного",
45,Pic-RUS_03-m_Ski-R.zip,знакомым мне здесь рассказали одну смешную и п...,"нагорных, доскачет, поехать, наложением, слышь","лыжах нагорных, дороге доскачет, позвать поеха...",горный,нагорных,"нагорных, доскачет, поехать, наложением, слышь",


In [75]:
trans_alpha.to_excel('trans_alpha_mark_ch.xlsx')

In [76]:
%time
trans_abk['mistakes'] = ''
trans_abk['bigram_mist'] = ''
trans_abk['new_words'] = ''
trans_abk['mistakes_1st_type'] = ''
trans_abk['mistakes_2nd_type'] = ''
trans_abk['absent_words'] = ''


for row in trans_abk.abk_transcripts.index:
    print('Обрабатываю ряд ', row)
    text = insert_hyphen_in_text(trans_abk.abk_transcripts[row], hyphened_words)
    mistakes, bigrams_prob_dict, no_such_word = find_mistakes(text, cfreq, cprob)
    true_mistakes = find_true_mistakes(mistakes, bigrams_prob_dict)
    leven_mist = find_variation(vocab, true_mistakes)
    
    first_mistakes, second_mistakes, new_words = categorize_mistakes(leven_mist, true_mistakes, cprob)
    
    trans_abk['mistakes'][row] = ', '.join([key for key in true_mistakes])
    trans_abk['bigram_mist'][row] = ', '.join([true_mistakes[key] for key in true_mistakes])
    trans_abk['new_words'][row] =', '.join(new_words)
    trans_abk['absent_words'][row] = ', '.join(no_such_word)
    trans_abk['mistakes_1st_type'][row] = ', '.join(first_mistakes)
    trans_abk['mistakes_2nd_type'][row] = ', '.join(second_mistakes)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.82 µs
Обрабатываю ряд  35
	Всего ошибок:  10  из  148
	Количество биграм с отсутствующим в словаре словом:  2
Обрабатываю ряд  36
	Всего ошибок:  21  из  135
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  37
	Всего ошибок:  6  из  94
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  38
	Всего ошибок:  20  из  87
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  39
	Всего ошибок:  11  из  87
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  40
	Всего ошибок:  17  из  141
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  41
	Всего ошибок:  3  из  51
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  42
	Всего ошибок:  9  из  64
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  43
	Всего ошибок:  20  из  116
	Количество биграм с отсутствующим в словаре словом:  2
Обра

In [77]:
trans_abk.head()

,audio_ID,abk_transcripts,mistakes,bigram_mist,new_words,mistakes_1st_type,mistakes_2nd_type,absent_words
35,Pic-RUS_01-f_Pr-R.zip,жил был один дяденька его жены скоро должно бы...,"дяденька, снова","один дяденька, осталось снова","дядька, новый","дяденька, снова","дяденька, снова","один дячка, дячка заявил"
36,Pic-RUS_01-f_Pr-T.zip,товарищ тут с ним случилось проблема у его жен...,"товарищ, какие-нибудь, присоветует, ильич, нот...","товарищ тут, какое-нибудь какие-нибудь, думаю ...","товар, посоветовать","товарищ, присоветует","товарищ, какие-нибудь, присоветует, ильич, нот...",
37,Pic-RUS_01-f_Ski-R.zip,жил был один молодой человек это молодой челов...,,,,,,
38,Pic-RUS_01-f_Ski-T.zip,день жизнь одного точне увлекающейся спорту му...,"точне, товарищ, человека, заряд, нажрался, мил...","одного точне, спортивный товарищ, выпиской чел...","точный, вложить","точне, ложили","точне, товарищ, человека, заряд, нажрался, мил...",
39,Pic-RUS_02-f_Pr-R.zip,однозначно был день рождения её муж решил пода...,"грушой, машинку","маленькой грушой, грушой машинку",игрушка,грушой,"грушой, машинку",


In [78]:
trans_abk.to_excel('trans_abk_mark_ch.xlsx')

## Оценка алгоритма (только марковская цепь, с учетом вводных слов и дискурсивных маркеров)

### Разметка ошибок и подсчет метрик precision, recall, f1

Для оценки было размечено по 10 транскрипций для каждого транскрибатора, то есть проведем оценку качества по 20 транскрибациям

Выгрузка файлов для разметки

In [79]:
score_alpha = pd.DataFrame({'word':[], 'true':0, 'pred':0})
for ind in [35, 38, 39, 45, 46, 62, 64, 65, 66, 88]:
    score = pd.DataFrame({'word':trans_alpha.alphacep_transcripts[ind].split()})
    score['true'] = 0
    score['pred'] = 0
    score.to_excel(str(ind)+'score_alpha_mark_ch.xlsx')

In [80]:
score_abk = pd.DataFrame({'word':[], 'true':0, 'pred':0})
for ind in [35, 38, 39, 45, 46, 62, 64, 65, 66, 88]:
    score = pd.DataFrame({'word':trans_abk.abk_transcripts[ind].split()})
    score['true'] = 0
    score['pred'] = 0
    score.to_excel(str(ind)+'score_abk_mark_ch.xlsx')

Загрузка размеченных файлов

In [90]:
files_alpha = ['35score_alpha_mark_ch.xlsx','38score_alpha_mark_ch.xlsx','39score_alpha_mark_ch.xlsx','45score_alpha_mark_ch.xlsx','46score_alpha_mark_ch.xlsx', '62score_alpha_mark_ch.xlsx','64score_alpha_mark_ch.xlsx','65score_alpha_mark_ch.xlsx','66score_alpha_mark_ch.xlsx','88score_alpha_mark_ch.xlsx']
files_abk = ['35score_abk_mark_ch.xlsx','38score_abk_mark_ch.xlsx','39score_abk_mark_ch.xlsx','45score_abk_mark_ch.xlsx','46score_abk_mark_ch.xlsx', '62score_abk_mark_ch.xlsx','64score_abk_mark_ch.xlsx','65score_abk_mark_ch.xlsx','66score_abk_mark_ch.xlsx','88score_abk_mark_ch.xlsx']

Формирование значений y_true для транскрибаций alpha

In [91]:
y_true_alpha = []
y_pred_alpha = []
for file in files_alpha:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_true_alpha+=y_true 
    y_pred = list(df.pred)
    y_pred_alpha += y_pred

Формирование значений y_true для транскрибаций abk

In [92]:
y_true_abk = []
y_pred_abk = []
for file in files_abk:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_true_abk+=y_true 
    y_pred = list(df.pred)
    y_pred_abk += y_pred

#### Подсчет метрик для всех транскрибаций

In [93]:
print('For all alphacep transcriptions:')
print('\tPrecision: {:0.4f}'.format(precision_score(y_true_alpha, y_pred_alpha)))
print('\tRecall: {:0.4f}'.format(recall_score(y_true_alpha, y_pred_alpha)))
print('\tF1: {:0.4f}'.format(f1_score(y_true_alpha, y_pred_alpha)))
print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true_alpha, y_pred_alpha)))

For all alphacep transcriptions:
	Precision: 0.6022
	Recall: 0.3889
	F1: 0.4726
	Accuracy: 0.8751


In [94]:
print('For all ABK transcriptions:')
print('\tPrecision: {:0.4f}'.format(precision_score(y_true_abk, y_pred_abk)))
print('\tRecall: {:0.4f}'.format(recall_score(y_true_abk, y_pred_abk)))
print('\tF1: {:0.4f}'.format(f1_score(y_true_abk, y_pred_abk)))
print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true_abk, y_pred_abk)))

For all ABK transcriptions:
	Precision: 0.4493
	Recall: 0.2138
	F1: 0.2897
	Accuracy: 0.8433


#### Подсчет метрик отдельно для кажой транскрибации

In [95]:
for file in files_alpha:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_pred = list(df.pred)
    print('For file:', file)
    print('\tPrecision: {:0.4f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:0.4f}'.format(recall_score(y_true, y_pred)))
    print('\tF1: {:0.4f}'.format(f1_score(y_true, y_pred)))
    print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true, y_pred)))

For file: 35score_alpha_mark_ch.xlsx
	Precision: 0.6364
	Recall: 0.4375
	F1: 0.5185
	Accuracy: 0.9091
For file: 38score_alpha_mark_ch.xlsx
	Precision: 0.6667
	Recall: 0.8571
	F1: 0.7500
	Accuracy: 0.9529
For file: 39score_alpha_mark_ch.xlsx
	Precision: 1.0000
	Recall: 0.5000
	F1: 0.6667
	Accuracy: 0.9630
For file: 45score_alpha_mark_ch.xlsx
	Precision: 0.8000
	Recall: 0.6667
	F1: 0.7273
	Accuracy: 0.9651
For file: 46score_alpha_mark_ch.xlsx
	Precision: 0.2500
	Recall: 0.1429
	F1: 0.1818
	Accuracy: 0.8714
For file: 62score_alpha_mark_ch.xlsx
	Precision: 0.5000
	Recall: 0.2222
	F1: 0.3077
	Accuracy: 0.9143
For file: 64score_alpha_mark_ch.xlsx
	Precision: 0.4000
	Recall: 0.3158
	F1: 0.3529
	Accuracy: 0.7982
For file: 65score_alpha_mark_ch.xlsx
	Precision: 0.8750
	Recall: 0.2188
	F1: 0.3500
	Accuracy: 0.7778
For file: 66score_alpha_mark_ch.xlsx
	Precision: 0.5000
	Recall: 0.3077
	F1: 0.3810
	Accuracy: 0.8687
For file: 88score_alpha_mark_ch.xlsx
	Precision: 0.6154
	Recall: 0.5517
	F1: 0.581

In [96]:
for file in files_abk:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_pred = list(df.pred)
    print('For file:', file)
    print('\tPrecision: {:0.4f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:0.4f}'.format(recall_score(y_true, y_pred)))
    print('\tF1: {:0.4f}'.format(f1_score(y_true, y_pred)))
    print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true, y_pred)))

For file: 35score_abk_mark_ch.xlsx
	Precision: 0.6667
	Recall: 0.1429
	F1: 0.2353
	Accuracy: 0.9167
For file: 38score_abk_mark_ch.xlsx
	Precision: 0.4444
	Recall: 0.6667
	F1: 0.5333
	Accuracy: 0.9231
For file: 39score_abk_mark_ch.xlsx
	Precision: 0.5000
	Recall: 0.0667
	F1: 0.1176
	Accuracy: 0.8333
For file: 45score_abk_mark_ch.xlsx
	Precision: 0.7500
	Recall: 0.3750
	F1: 0.5000
	Accuracy: 0.9310
For file: 46score_abk_mark_ch.xlsx
	Precision: 0.2857
	Recall: 0.2500
	F1: 0.2667
	Accuracy: 0.8281
For file: 62score_abk_mark_ch.xlsx
	Precision: 0.0000
	Recall: 0.0000
	F1: 0.0000
	Accuracy: 0.9029
For file: 64score_abk_mark_ch.xlsx
	Precision: 0.3000
	Recall: 0.2143
	F1: 0.2500
	Accuracy: 0.8235
For file: 65score_abk_mark_ch.xlsx
	Precision: 0.7500
	Recall: 0.1111
	F1: 0.1935
	Accuracy: 0.7596
For file: 66score_abk_mark_ch.xlsx
	Precision: 0.5455
	Recall: 0.3158
	F1: 0.4000
	Accuracy: 0.8022
For file: 88score_abk_mark_ch.xlsx
	Precision: 0.4375
	Recall: 0.2593
	F1: 0.3256
	Accuracy: 0.6463


# Пробуем устранить количество ложно распознанных ошибок с помощью векторной модели 

## Загрузка готовой векторной модели с сайта RusVectores

Зазгрузим модель tayga-func_upos_skipgram_300_5_2019, обученную на корпусе Taiga c функциональными словами.

Ссылка на модель находится [здесь]('http://vectors.nlpl.eu/repository/11/186.zip'): 

**Характеристики модели:**

Дата создания  - январь 2019

Размер обучающего корпуса - почти 5 млрд слов 

Объем словаря - 249 946 слов

Алгоритм обучения - Continuous Skipgram

Размер окна - 5

Размерность вектора - 300

Набор частеречных тэгов - Universal Tags

In [97]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/186.zip' #ссылка на модель
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    w2v_tayga_model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Небольшой тест модели:

In [98]:
words = ['дяденька_NOUN', 'зимний_ADJ']

In [99]:
for word in words:
    if word in w2v_tayga_model: #проверяем, есть ли слово в модели
        for i in w2v_tayga_model.most_similar(positive=[word], topn=20): # выдаем 10 ближайших соседей слова:
            if re.search(r'_[A-Z]{3,5}', i[0]) is not None: 
                pos_tag = re.search(r'_[A-Z]{1,5}', i[0])[0]
                print(i[0], i[1]) # слово + коэффициент косинусной близости
                print(pos_tag)
                print('\n')
    else:
        print(word + ' is not present in the model')

тетенька_NOUN 0.8230295777320862
_NOUN


дядечка_NOUN 0.8126401305198669
_NOUN


дядька_NOUN 0.6924134492874146
_NOUN


дяденька_ADJ 0.6899006962776184
_ADJ


тетенька_ADJ 0.6767088174819946
_ADJ


тетечка_NOUN 0.654828667640686
_NOUN


дяденька_PROPN 0.6515356302261353
_PROPN


старичок_NOUN 0.6482418179512024
_NOUN


тетенек_NOUN 0.6351189613342285
_NOUN


мужичок_NOUN 0.627760648727417
_NOUN


дедок_NOUN 0.5976375341415405
_NOUN


тётечка_NOUN 0.5941250324249268
_NOUN


дедуля_NOUN 0.5877871513366699
_NOUN


мужичка_NOUN 0.5871917009353638
_NOUN


тетенький_NOUN 0.5848380327224731
_NOUN


мальчик_NOUN 0.578131914138794
_NOUN


дяденек_NOUN 0.5729660987854004
_NOUN


толстячок_NOUN 0.5715343952178955
_NOUN


мужик_NOUN 0.5627331137657166
_NOUN


старикан_NOUN 0.5626460313796997
_NOUN


зимний_NOUN 0.8504959344863892
_NOUN


осенний_ADJ 0.730405330657959
_ADJ


летний_ADJ 0.7238248586654663
_ADJ


весенний_ADJ 0.6703113317489624
_ADJ


зимний_PRON 0.6563525795936584
_PRON


зима_NOUN 

Нам нужно будет использовать тот же набор частеречных тэгов в тестовом корпусе, что используется и в обучающем, то есть тэги Universal Dependencies.

Несколько слов, почему мы не выбросили частеречные тэги из предобученной векторной модели:
    1. Отсеиваем сомнительные случаи (незнакомые слова), скажем слова с тэгами _X.
    2. Вычисляем вероятность в нашей марковской цепи только для тех семантических ассоциатов, у которых частеречный тэг совпадает с тэгом у слова-ошибки в нашей биграмме
    3. Исключаем частеречную омонимию

## Создание словаря для конвертации частеречных тэгов 
[Pymorphy2](http://opencorpora.org/dict.php?act=gram&order=priority) 
## в тэги 
[Unversal POS](https://universaldependencies.org/u/pos/all.html)

In [101]:
mapping_Pymorphy_UPos = {
'ADJF': 'ADJ',
'ADJS': 'ADJ',
'ADVB': 'ADV',
'Apro': 'DET',
'COMP': 'ADV',  # может быть и прилагательным, недостаточно информации о тэге в opencorpora
'CONJ': 'CONJ',
'GRND': 'VERB',
'INFN': 'VERB',
'INTJ': 'INTJ',
'NOUN': 'NOUN',
'NPRO': 'PRON',
'NUMR': 'NUM',
'NUMB': 'NUM',
'PART': 'PRCL',
'PNCT': 'PUNCT',
'PRCL': 'PART',
'PREP': 'ADP',
'PRTF': 'VERB',
'PRTS': 'VERB',
'VERB': 'VERB'
}

In [102]:
#Сохранение словаря для конвертации частеречных тэгов
save_mapping = open("map_pymorphy2_UPos.pickle" , "wb" )
pickle.dump(mapping_Pymorphy_UPos, save_mapping)
save_mapping .close()

[Ссылка]('https://github.com/smekur/Spoken_Corpora_with_Kaldi/blob/master/mistakes_search/others/map_pymorphy2_UPos.pickle') для скачивания словаря конвертации тэгов выше

### Объявление функций для работы с новым алгоритмом на основе векторной модели Word2Vec

In [103]:
'''
Функция осуществляет частеречную разметку в виде Universal POS-tags у предполагаемого слова-ошибки. 
(Предобработка необходима для работы с предобученной моделью от RusVectores.)

Аргументы:
    true_mistakes - словарь вида: {`слово-ошибка`: `биграмма-кандидат`}
    mapping - словарь-таблица для конвертации тэгов частей речи из одного формата в другой.  
Вывод:
    tagged_true_mistakes - словарь вида: {`слово-ошибка_с частеречным тэгом`: `биграма-кандидат`}
'''

def insert_pos_tags(true_mistakes, mapping):
    tagged_true_mistakes = {}
    for mistake in true_mistakes: #{'дяденька': 'один дяденька'}
        #print('Итерируемся по словарю ошибок. Берем ключ словаря(ошибку):', mistake, '\n')
        mistake_analized = morph.parse(mistake) # морф анализ
        #print('Делаем морф анализ. ключа. Результат:', mistake_analized, '\n')
        mistake_lemma = mistake_analized[0].normal_form #извлекаем лемму
        #print('Печатаем лемму ключа:', mistake_lemma, '\n')
        pos = mistake_analized[0].tag.POS #извлекаем частеречный тэг
        #print('Вычисляем частеречный тэг ключа:', pos, '\n')
        if pos in mapping: #если частеречный тэг есть в словаре конвертации тэгов
            tagged_lemma = mistake_lemma + '_' + mapping[pos] #получаем лемму с частеречным тэгом: lemma_POS-tag
            #print('Тэг в мэппинге. Печатаю лемму ключа с тэгом:', tagged_lemma, '\n')
        else:
            tagged_lemma = mistake_lemma + '_X' # на случай, если попадется тэг, которого нет в словаре
            #print('Нет тэга в мэппинге. Печатаю лемму ключа с тэгом X:', tagged_lemma, '\n')
        value = true_mistakes.get(mistake) 
        #print('Значение для ключа в новом словаре:', value, '\n')
        tagged_true_mistakes[tagged_lemma] = value
        #print('Пополняем новый словарь. Добавляем лемму с тэгом в виде ключа', tagged_lemma,  'и его значение:', value)
    
    return tagged_true_mistakes

In [108]:
'''
Функция для проверки слов-ошибок с помощью косинусной близости и модели word2vec: 
Проверяем с помощью косинусной близости, есть ли такое близкое слову-ошибке слово, которое образует биграмму, вероятность у которой в обученной марковской цепи выше нуля.
Если из 10 ближайших соседей такое слово находится, то мы не считаем слово-ошибку в биграмме истинной ошибкой. 

Аргументы:
    tagged_true_mistakes - словарь вида: {`слово-ошибка_с частеречным тэгом`: `биграмма со словом-ошибкой`}.
    vec_model - векторная модель

Вывод:
    not_mistakes - список отфильтрованных ошибок: такие слова, которые считались ошибками в марковской цепи, а после применения алгоритма с косинусной близостью были отфильтрованы
    no_UDPos_tags - список слов, у которых частеречный тэг не имеет соответствия в наборе тэгов UDPos
    filtered_true_mistakes - словарь истинных ошибок после отработки алгоритма  с косинусной близостью: ключ без частеречного тэга и в виде исходной словоформы текста
'''

def filter_mistakes_with_cos_similarity(tagged_true_mistakes, vec_model, model_prob, true_mistakes):
    
    no_UDPos_tags = [] 
    filtered_true_mistakes = {} 
    not_mistakes = []

    for key in tagged_true_mistakes.copy():
    #print('Печатаю ключ в исходном словаре:', key)
        if key in vec_model: #проверяем, есть ли слово в модели
            print('Ключ есть в модели word2vec')
            if re.search(r'_[A-Z]{3,5}', key) is not None: #проверяем, что есть частеречный тэг не X
                key_tag = re.search(r'_[A-Z]{3,5}', key)[0] #запоминаем частеречный тэг у слова-ошибки
                key_without_tag = re.sub(r'_[A-Z]{3,5}','', key) #запоминаем слово-ошибку без частеречного тэга
                #print('Печатаю частеречный тэг ключа:', key_tag)
                #print('Печатаю ключ без тэга:', key_without_tag)
                value = tagged_true_mistakes.get(key) #берем значение ключа
                #print('Значение нашего ключа:', value)
                big = value.split() #разбиваем на части нашу строку из двух слов
                big = [morph.normal_forms(token)[0] for token in big] #лемматизируем каждое слово в биграмме # ['ранний', 'зимний']
            else: 
                print('У слова-ошибки обнаружен частеречный тэг, который отсутствует в UDPos') 
                no_UDPos_tags.append(key) #если тэг _X, то заносим такие слова в отдельный список
            for candidate in vec_model.most_similar(positive=[key], topn=10): #смотрим 10 ближайших соседей для слова-ошибки:
                #print('Итерируемся по соседям ключа. Берем соседа:', candidate)
                if re.search(r'_[A-Z]{3,5}', candidate[0]) is not None: #не рассматриваем кандидатов с тэгами _X
                    candidate_tag = re.search(r'_[A-Z]{3,5}', candidate[0])[0] #запоминаем ключ кандидата 
                if re.fullmatch(r'[а-я]+_[A-Z]{3,5}',candidate[0]) is not None and candidate[0]!= key and candidate_tag == key_tag: 
                #берем только кандидатов, которые отвечают формату: слово_POStag; не совпадают с нашим ключом и частеречный тэг у которого совпадает с тэгом у нашего ключа
                    #print('Кандидат отвечает формату:', candidate[0])
                    #print('Тэг кандидата:', candidate_tag)
                    candidate_without_tag = re.sub(r'_[A-Z]{3,5}','', candidate[0]) #убираем частеречный тэг у подходящего кандитата
                    #print('Кандидат без тэга:', candidate_without_tag)
                
                    if key_without_tag == big[1]:
                        #print("Ключ-ошибка", key_without_tag, "является вторым словом в биграме:", big, 'Подставляю кандидата', candidate_without_tag, 'в .prob')
                        if model_prob[big[0]].prob(candidate_without_tag)<=0:
                            #print('Кандидат',candidate_without_tag, 'не образует вероятностную биграмму')
                            continue
                        else: #model_prob[big[0]].prob(candidate_without_tag)>0: #если мы поймали вероятностную биграмму 
                            print('Слово', key, 'не ошибка')
                            not_mistakes.append(key)
                            del tagged_true_mistakes[key] #удаляем ключ со словом ошибкой из словаря
                            break
                        
                    else:
                        #print("Ключ-ошибка", key_without_tag, "является первым словом в биграме:", big)
                        if model_prob[candidate_without_tag].prob(big[1])<=0:
                            #print('Кандидат',candidate_without_tag, 'не образует вероятностную биграмму')
                            continue
                        else: #model_prob[candidate_without_tag].prob(big[1])>0:
                            #print('Слово', key, 'не ошибка')
                            not_mistakes.append(key)
                            del tagged_true_mistakes[key]
                            break
    
    filtered_true_mistakes = {} #создаем список ошибок после фильтрации алгоритмом с помощью векторной модели
    
    for key in true_mistakes:
        if true_mistakes.get(key) in tagged_true_mistakes.values():
            filtered_true_mistakes[key] = true_mistakes.get(key) 
                            
    return filtered_true_mistakes, not_mistakes, no_UDPos_tags        

## Работа с транскрибациями (Word2Vec)

In [109]:
#Снова загрузим наши транскрибации (тестовый корпус):
trans = pd.read_excel('all_transcripts.xlsx')
trans_alpha = trans.loc[
       (trans.audio_ID.str.contains('Sib')|trans.audio_ID.str.contains('Pic'))&trans.alphacep_transcripts.notnull(),
       ['audio_ID','alphacep_transcripts']
    ]
trans_abk = trans.loc[
       (trans.audio_ID.str.contains('Sib')|trans.audio_ID.str.contains('Pic'))&trans.abk_transcripts.notnull(),
       ['audio_ID','abk_transcripts']
    ]

#### Корпус транскрибатора Alphacep

In [110]:
%time

trans_alpha['mistakes'] = ''
trans_alpha['bigram_mist'] = ''
trans_alpha['new_words'] = ''
trans_alpha['mistakes_1st_type'] = ''
trans_alpha['mistakes_2nd_type'] = ''
trans_alpha['absent_words'] = ''

trans_alpha['w2v_mistakes'] = ''
trans_alpha['w2v_bigram_mist'] = ''
trans_alpha['w2v_new_words'] = ''
trans_alpha['w2v_mistakes_1st_type'] = ''
trans_alpha['w2v_mistakes_2nd_type'] = ''


for row in trans_alpha.alphacep_transcripts.index:
    print('Обрабатываю ряд ', row)
    text = insert_hyphen_in_text(trans_alpha.alphacep_transcripts[row], hyphened_words)
    mistakes, bigrams_prob_dict, no_such_word = find_mistakes(text, cfreq, cprob)
    true_mistakes = find_true_mistakes(mistakes, bigrams_prob_dict)
    leven_mist = find_variation(vocab, true_mistakes)
    
    first_mistakes, second_mistakes, new_words = categorize_mistakes(leven_mist, true_mistakes, cprob)
    
    tagged_true_mistakes = insert_pos_tags(true_mistakes, mapping_Pymorphy_UPos)
    filtered_true_mistakes, not_mistakes, no_UDPos_tags  = filter_mistakes_with_cos_similarity(tagged_true_mistakes, w2v_tayga_model, cprob, true_mistakes)
    w2v_leven_mist = find_variation(vocab, filtered_true_mistakes)
    
    
    w2v_first_mistakes, w2v_second_mistakes, w2v_new_words = categorize_mistakes(w2v_leven_mist, filtered_true_mistakes, cprob)
    
    trans_alpha['mistakes'][row] = ', '.join([key for key in true_mistakes])
    trans_alpha['bigram_mist'][row] = ', '.join([true_mistakes[key] for key in true_mistakes])
    trans_alpha['new_words'][row] =', '.join(new_words)
    trans_alpha['absent_words'][row] = ', '.join(no_such_word)
    trans_alpha['mistakes_1st_type'][row] = ', '.join(first_mistakes)
    trans_alpha['mistakes_2nd_type'][row] = ', '.join(second_mistakes)
    
    trans_alpha['w2v_mistakes'][row] = ', '.join([key for key in filtered_true_mistakes])
    trans_alpha['w2v_bigram_mist'][row] = ', '.join([filtered_true_mistakes[key] for key in filtered_true_mistakes])
    trans_alpha['w2v_new_words'][row] =', '.join(w2v_new_words)
    trans_alpha['w2v_mistakes_1st_type'][row] = ', '.join(w2v_first_mistakes)
    trans_alpha['w2v_mistakes_2nd_type'][row] = ', '.join(w2v_second_mistakes)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs
Обрабатываю ряд  35
	Всего ошибок:  24  из  137
	Количество биграм с отсутствующим в словаре словом:  2
Ключ есть в модели word2vec
Слово дяденька_NOUN не ошибка
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Слово наставить_VERB не ошибка
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Обрабатываю ряд  38
	Всего ошибок:  15  из  81
	Количество биграм с отсутствующим в словаре словом:  3
Ключ есть в модели word2vec
Слово хотеться_VERB не ошибка
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Обрабатываю ряд  39
	Всего ошибок:  12  из  78
	Количество биграм с отсутствующим в словаре словом:  0
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Слово посылать_VERB не ошиб

In [111]:
trans_alpha.head()

,audio_ID,alphacep_transcripts,mistakes,bigram_mist,new_words,mistakes_1st_type,mistakes_2nd_type,absent_words,w2v_mistakes,w2v_bigram_mist,w2v_new_words,w2v_mistakes_1st_type,w2v_mistakes_2nd_type
35,Pic-RUS_01-f_Pr-R.zip,жилбыл один дяденька по его жены скоро должно ...,"дяденька, плот, ночного, посол, салон, наставь...","один дяденька, случится плот, дяденька ночного...","дядька, ставить","дяденька, наставь","дяденька, плот, ночного, посол, салон, наставь...","посол варт, варт салон","плот, ночного, посол, дядечка, носок, маменьку...","случится плот, дяденька ночного, глупости посо...",,,"плот, ночного, посол, дядечка, носок, маменьку..."
38,Pic-RUS_01-f_Ski-T.zip,генин жизни одного очень увлекающийся спортом ...,"хочется, партийный, товарищ, тоесть, природу, ...","катался хочется, хочется партийный, партийный ...",хотеть,хочется,"хочется, партийный, товарищ, тоесть, природу, ...","генин жизни, и норг, норг тоесть","партийный, товарищ, тоесть, природу, ходит, не...","хочется партийный, партийный товарищ, тоесть п...",,,"партийный, товарищ, тоесть, природу, ходит, не..."
39,Pic-RUS_02-f_Pr-R.zip,однозначным был день рождения мышь решил подар...,"однозначным, мышь, посылала","однозначным был, рождения мышь, в посылала","однозначно, подослать","однозначным, посылала","однозначным, мышь, посылала",,мышь,рождения мышь,,,мышь
42,Pic-RUS_02-f_Ski-T.zip,этот человек встал рано утром позавтракал а по...,"наложены, божественного","отправился наложены, голову божественного",,,"наложены, божественного",,"наложены, божественного","отправился наложены, голову божественного",,,"наложены, божественного"
45,Pic-RUS_03-m_Ski-R.zip,знакомым мне здесь рассказали одну смешную и п...,"нагорных, доскачет, поехать, наложением, слышь","лыжах нагорных, дороге доскачет, позвать поеха...",горный,нагорных,"нагорных, доскачет, поехать, наложением, слышь",,"нагорных, поехать, наложением, слышь","лыжах нагорных, позвать поехать, поехать налож...",горный,нагорных,"нагорных, поехать, наложением, слышь"


In [112]:
trans_alpha.to_excel('trans_alpha_w2v.xlsx')

In [113]:
trans_alpha.tail()

,audio_ID,alphacep_transcripts,mistakes,bigram_mist,new_words,mistakes_1st_type,mistakes_2nd_type,absent_words,w2v_mistakes,w2v_bigram_mist,w2v_new_words,w2v_mistakes_1st_type,w2v_mistakes_2nd_type
84,Sib_12-f,так оно и какуюнибудь игру сработает о чем не ...,"киберов, паники, пайки, пупком, спектры, рубан...","не киберов, собирали паники, собирали пайки, м...","кибермэн, сектор, амулет, собрать, дом, побежать","киберов, спектры, гамлета, собирать, домовым, ...","киберов, паники, пайки, пупком, спектры, рубан...","пупком тягнуть, тягнуть спектры, не аничковом,...","паники, пупком, ибо, щурясь, партнер, гамлета,...","собирали паники, мать пупком, рубанула ибо, по...",амулет,гамлета,"паники, пупком, ибо, щурясь, партнер, гамлета,..."
85,Sib_13-f,родились мы с братом тысяча девятьсот сороково...,"недолго, холод, вани, бедно, крапиву, созерцал...","мальчика недолго, что холод, у вани, встали бе...","долго, холодный, ани, принимать, светить, напо...","недолго, холод, вани, принималась, сети, напои...","недолго, холод, вани, бедно, крапиву, созерцал...","принималась гарбуза, гарбуза коровушка, питали...","холод, вани, бедно, крапиву, созерцала, сети, ...","что холод, у вани, встали бедно, нам крапиву, ...","холодный, ани, светить, напомнить, чёткий","холод, вани, сети, напои, сеткой","холод, вани, бедно, крапиву, созерцала, сети, ..."
87,Sib_15-f,угу лошади и нас эвакуировали будь я в маленьк...,"угу, катуни, таку, бурною, цветению, глуше, че...","угу лошади, его катуни, соки таку, берег бурно...","каунтить, такой, что, потихоньку, правильно, а...","катуни, таку, чем, потихонечку, правильных, ал...","угу, катуни, таку, бурною, цветению, глуше, че...","там пятигорске, пятигорске в, было багульник, ...","угу, катуни, таку, бурною, цветению, глуше, че...","угу лошади, его катуни, соки таку, берег бурно...","каунтить, такой, что, потихоньку, правильно, м...","катуни, таку, чем, потихонечку, правильных, мо...","угу, катуни, таку, бурною, цветению, глуше, че..."
88,Sib_16-m,с четырнадцати лет я начал работать работал пр...,"прицеп, комбайне, комбайн, надувался, рецептур...","работал прицеп, поводу комбайне, комбайне комб...","прицел, сук","прицеп, сучка","прицеп, комбайне, комбайн, надувался, рецептур...","надувался коммунар, коммунар рецептура, леспро...","прицеп, надувался, рецептура, комбайна, бароме...","работал прицеп, комбайн надувался, рецептура к...",прицел,прицеп,"прицеп, надувался, рецептура, комбайна, бароме..."
89,Sib_17-m,ты хоть три сюжета три сюжета а которое я помо...,"ва, спи, воронежа, штанцы, окошек, отара, почт...","было ва, воронеж спи, окраине воронежа, малень...","танец, тара, один, полуночь, отскочить, хлеб","штанцы, отара, она, полночь, отмочить, хлебная","ва, спи, воронежа, штанцы, окошек, отара, почт...","три масючка, масючка была, стоял нафту, нафту ...","ва, спи, воронежа, окошек, отара, почти, бирж,...","было ва, воронеж спи, окраине воронежа, штанцы...","тара, полуночь","отара, полночь","ва, спи, воронежа, окошек, отара, почти, бирж,..."


#### Корпус транскрибатора АБК

In [114]:
%time
trans_abk['mistakes'] = ''
trans_abk['bigram_mist'] = ''
trans_abk['new_words'] = ''
trans_abk['mistakes_1st_type'] = ''
trans_abk['mistakes_2nd_type'] = ''
trans_abk['absent_words'] = ''

trans_abk['w2v_mistakes'] = ''
trans_abk['w2v_bigram_mist'] = ''
trans_abk['w2v_new_words'] = ''
trans_abk['w2v_mistakes_1st_type'] = ''
trans_abk['w2v_mistakes_2nd_type'] = ''


for row in trans_abk.abk_transcripts.index:
    print('Обрабатываю ряд ', row)
    text = insert_hyphen_in_text(trans_abk.abk_transcripts[row], hyphened_words)
    mistakes, bigrams_prob_dict, no_such_word = find_mistakes(text, cfreq, cprob)
    true_mistakes = find_true_mistakes(mistakes, bigrams_prob_dict)
    leven_mist = find_variation(vocab, true_mistakes)
    
    first_mistakes, second_mistakes, new_words = categorize_mistakes(leven_mist, true_mistakes, cprob)
    
    tagged_true_mistakes = insert_pos_tags(true_mistakes, mapping_Pymorphy_UPos)
    filtered_true_mistakes, not_mistakes, no_UDPos_tags  = filter_mistakes_with_cos_similarity(tagged_true_mistakes, w2v_tayga_model, cprob, true_mistakes)
    w2v_leven_mist = find_variation(vocab, filtered_true_mistakes)
    
    
    w2v_first_mistakes, w2v_second_mistakes, w2v_new_words = categorize_mistakes(w2v_leven_mist, filtered_true_mistakes, cprob)
    
    trans_abk['mistakes'][row] = ', '.join([key for key in true_mistakes])
    trans_abk['bigram_mist'][row] = ', '.join([true_mistakes[key] for key in true_mistakes])
    trans_abk['new_words'][row] =', '.join(new_words)
    trans_abk['absent_words'][row] = ', '.join(no_such_word)
    trans_abk['mistakes_1st_type'][row] = ', '.join(first_mistakes)
    trans_abk['mistakes_2nd_type'][row] = ', '.join(second_mistakes)
    
    trans_abk['w2v_mistakes'][row] = ', '.join([key for key in filtered_true_mistakes])
    trans_abk['w2v_bigram_mist'][row] = ', '.join([filtered_true_mistakes[key] for key in filtered_true_mistakes])
    trans_abk['w2v_new_words'][row] =', '.join(w2v_new_words)
    trans_abk['w2v_mistakes_1st_type'][row] = ', '.join(w2v_first_mistakes)
    trans_abk['w2v_mistakes_2nd_type'][row] = ', '.join(w2v_second_mistakes)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 10 µs
Обрабатываю ряд  35
	Всего ошибок:  10  из  148
	Количество биграм с отсутствующим в словаре словом:  2
Ключ есть в модели word2vec
Слово дяденька_NOUN не ошибка
Ключ есть в модели word2vec
Обрабатываю ряд  36
	Всего ошибок:  21  из  135
	Количество биграм с отсутствующим в словаре словом:  0
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Слово илья_NOUN не ошибка
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Обрабатываю ряд  37
	Всего ошибок:  6  из  94
	Количество биграм с отсутствующим в словаре словом:  0
Обрабатываю ряд  38
	Всего ошибок:  20  из  87
	Количество биграм с отсутствующим в словаре словом:  0
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модели word2vec
Ключ есть в модел

In [115]:
trans_abk.head()

,audio_ID,abk_transcripts,mistakes,bigram_mist,new_words,mistakes_1st_type,mistakes_2nd_type,absent_words,w2v_mistakes,w2v_bigram_mist,w2v_new_words,w2v_mistakes_1st_type,w2v_mistakes_2nd_type
35,Pic-RUS_01-f_Pr-R.zip,жил был один дяденька его жены скоро должно бы...,"дяденька, снова","один дяденька, осталось снова","дядька, новый","дяденька, снова","дяденька, снова","один дячка, дячка заявил",снова,осталось снова,новый,снова,снова
36,Pic-RUS_01-f_Pr-T.zip,товарищ тут с ним случилось проблема у его жен...,"товарищ, какие-нибудь, присоветует, ильич, нот...","товарищ тут, какое-нибудь какие-нибудь, думаю ...","товар, посоветовать","товарищ, присоветует","товарищ, какие-нибудь, присоветует, ильич, нот...",,"какие-нибудь, присоветует, нотариус, цен, прия...","какое-нибудь какие-нибудь, думаю присоветует, ...",посоветовать,присоветует,"какие-нибудь, присоветует, нотариус, цен, прия..."
37,Pic-RUS_01-f_Ski-R.zip,жил был один молодой человек это молодой челов...,,,,,,,,,,,
38,Pic-RUS_01-f_Ski-T.zip,день жизнь одного точне увлекающейся спорту му...,"точне, товарищ, человека, заряд, нажрался, мил...","одного точне, спортивный товарищ, выпиской чел...","точный, вложить","точне, ложили","точне, товарищ, человека, заряд, нажрался, мил...",,"точне, товарищ, человека, заряд, нажрался, мил...","одного точне, спортивный товарищ, выпиской чел...","точный, вложить","точне, ложили","точне, товарищ, человека, заряд, нажрался, мил..."
39,Pic-RUS_02-f_Pr-R.zip,однозначно был день рождения её муж решил пода...,"грушой, машинку","маленькой грушой, грушой машинку",игрушка,грушой,"грушой, машинку",,"грушой, машинку","маленькой грушой, грушой машинку",игрушка,грушой,"грушой, машинку"


In [116]:
trans_abk.tail()

,audio_ID,abk_transcripts,mistakes,bigram_mist,new_words,mistakes_1st_type,mistakes_2nd_type,absent_words,w2v_mistakes,w2v_bigram_mist,w2v_new_words,w2v_mistakes_1st_type,w2v_mistakes_2nd_type
75,Sib_03-m,да роман и девятнадцать лет родился в городе у...,"улан-удэ, здесь, геннадьевич, сидорова, сибиря...","городе улан-удэ, улан-удэ здесь, еще геннадьев...",,,"улан-удэ, здесь, геннадьевич, сидорова, сибиря...","этого шаликоевич, шаликоевич который","улан-удэ, здесь, геннадьевич, сибиряков, денеж...","городе улан-удэ, улан-удэ здесь, еще геннадьев...",,,"улан-удэ, здесь, геннадьевич, сибиряков, денеж..."
76,Sib_04-m,малики ругались на меня судя по всему год отра...,"малики, технолог, попросил, вычел, выдаче, тен...","малики ругались, мне технолог, технолог попрос...","технология, любопытный, пять, тащить, стартова...","технолог, опытных, опять, тащились, старт, поя...","малики, технолог, попросил, вычел, выдаче, тен...","тень альфия, альфия четвертой, опытных фрезеро...","малики, попросил, вычел, тень, григорий, опытн...","малики ругались, технолог попросил, деталь выч...","любопытный, стартовать, появляться","опытных, старт, появился","малики, попросил, вычел, тень, григорий, опытн..."
78,Sib_06-f,ламанову что я побежала в школу не понимала чт...,"лосева, чисел, потом","улице лосева, поймал чисел, чисел потом",простой,потом,"лосева, чисел, потом",ламанову что,"лосева, чисел","улице лосева, поймал чисел",,,"лосева, чисел"
86,Sib_14-f,третьего года начало самой дачный сезон живу с...,"живу, лето, грядах","сезон живу, ночи лето, меньше грядах","живой, этот","живу, лето","живу, лето, грядах","ухаживать варичем, варичем живут, удачный ураз...",живу,сезон живу,живой,живу,живу
88,Sib_16-m,сейчас силе я начал работать работу вообще на ...,"комбайне, комбайн, назывался, любые, коне, бог...","на комбайне, комбайне комбайн, комбайн называл...","любимый, конь, траут","любые, коне, тракта","комбайне, комбайн, назывался, любые, коне, бог...","назывался коммунар, коммунар любые, коне борон...","комбайн, назывался, любые, богатырский, леспро...","комбайне комбайн, комбайн назывался, любые пот...",любимый,любые,"комбайн, назывался, любые, богатырский, леспро..."


In [117]:
trans_abk.to_excel('trans_abk_w2v.xlsx')

## Оценка алгоритма c моделью Word2Vec

Выгрузка файлов для разметки

In [118]:
score_alpha = pd.DataFrame({'word':[], 'true':0, 'pred':0})
for ind in [35, 38, 39, 45, 46, 62, 64, 65, 66, 88]:
    score = pd.DataFrame({'word':trans_alpha.alphacep_transcripts[ind].split()})
    score['true'] = 0
    score['pred'] = 0
    score.to_excel(str(ind)+'score_alpha_w2vec.xlsx')

In [119]:
score_abk = pd.DataFrame({'word':[], 'true':0, 'pred':0})
for ind in [35, 38, 39, 45, 46, 62, 64, 65, 66, 88]:
    score = pd.DataFrame({'word':trans_abk.abk_transcripts[ind].split()})
    score['true'] = 0
    score['pred'] = 0
    score.to_excel(str(ind)+'score_abk_w2vec.xlsx')

Загрузка размеченных файлов

In [120]:
files_alpha = ['35score_alpha_w2vec.xlsx','38score_alpha_w2vec.xlsx','39score_alpha_w2vec.xlsx','45score_alpha_w2vec.xlsx','46score_alpha_w2vec.xlsx', '62score_alpha_w2vec.xlsx','64score_alpha_w2vec.xlsx','65score_alpha_w2vec.xlsx','66score_alpha_w2vec.xlsx','88score_alpha_w2vec.xlsx']
files_abk = ['35score_abk_w2vec.xlsx','38score_abk_w2vec.xlsx','39score_abk_w2vec.xlsx','45score_abk_w2vec.xlsx','46score_abk_w2vec.xlsx', '62score_abk_w2vec.xlsx','64score_abk_w2vec.xlsx','65score_abk_w2vec.xlsx','66score_abk_w2vec.xlsx','88score_abk_w2vec.xlsx']

Формирование значений y_true для транскрибаций alpha

In [121]:
y_true_alpha = []
y_pred_alpha = []
for file in files_alpha:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_true_alpha+=y_true 
    y_pred = list(df.pred)
    y_pred_alpha += y_pred

Формирование значений y_true для транскрибаций abk

In [122]:
y_true_abk = []
y_pred_abk = []
for file in files_abk:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_true_abk+=y_true 
    y_pred = list(df.pred)
    y_pred_abk += y_pred

#### Подсчет метрик для всех транскрибаций

In [123]:
print('For all alphacep transcriptions:')
print('\tPrecision: {:0.4f}'.format(precision_score(y_true_alpha, y_pred_alpha)))
print('\tRecall: {:0.4f}'.format(recall_score(y_true_alpha, y_pred_alpha)))
print('\tF1: {:0.4f}'.format(f1_score(y_true_alpha, y_pred_alpha)))
print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true_alpha, y_pred_alpha)))

For all alphacep transcriptions:
	Precision: 0.6301
	Recall: 0.3194
	F1: 0.4240
	Accuracy: 0.8751


In [124]:
print('For all ABK transcriptions:')
print('\tPrecision: {:0.4f}'.format(precision_score(y_true_abk, y_pred_abk)))
print('\tRecall: {:0.4f}'.format(recall_score(y_true_abk, y_pred_abk)))
print('\tF1: {:0.4f}'.format(f1_score(y_true_abk, y_pred_abk)))
print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true_abk, y_pred_abk)))

For all ABK transcriptions:
	Precision: 0.4912
	Recall: 0.1931
	F1: 0.2772
	Accuracy: 0.8495


#### Подсчет метрик отдельно для кажой транскрибации

In [125]:
for file in files_alpha:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_pred = list(df.pred)
    print('For file:', file)
    print('\tPrecision: {:0.4f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:0.4f}'.format(recall_score(y_true, y_pred)))
    print('\tF1: {:0.4f}'.format(f1_score(y_true, y_pred)))
    print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true, y_pred)))

For file: 35score_alpha_w2vec.xlsx
	Precision: 0.7500
	Recall: 0.3750
	F1: 0.5000
	Accuracy: 0.9161
For file: 38score_alpha_w2vec.xlsx
	Precision: 0.7143
	Recall: 0.7143
	F1: 0.7143
	Accuracy: 0.9529
For file: 39score_alpha_w2vec.xlsx
	Precision: 1.0000
	Recall: 0.1667
	F1: 0.2857
	Accuracy: 0.9383
For file: 45score_alpha_w2vec.xlsx
	Precision: 0.7500
	Recall: 0.5000
	F1: 0.6000
	Accuracy: 0.9535
For file: 46score_alpha_w2vec.xlsx
	Precision: 0.3333
	Recall: 0.1429
	F1: 0.2000
	Accuracy: 0.8857
For file: 62score_alpha_w2vec.xlsx
	Precision: 0.5000
	Recall: 0.2222
	F1: 0.3077
	Accuracy: 0.9143
For file: 64score_alpha_w2vec.xlsx
	Precision: 0.4167
	Recall: 0.2632
	F1: 0.3226
	Accuracy: 0.8073
For file: 65score_alpha_w2vec.xlsx
	Precision: 0.8333
	Recall: 0.1562
	F1: 0.2632
	Accuracy: 0.7607
For file: 66score_alpha_w2vec.xlsx
	Precision: 0.5714
	Recall: 0.3077
	F1: 0.4000
	Accuracy: 0.8788
For file: 88score_alpha_w2vec.xlsx
	Precision: 0.6667
	Recall: 0.4828
	F1: 0.5600
	Accuracy: 0.7925


In [126]:
for file in files_abk:
    df = pd.read_excel(file)
    y_true = list(df.true)
    y_pred = list(df.pred)
    print('For file:', file)
    print('\tPrecision: {:0.4f}'.format(precision_score(y_true, y_pred)))
    print('\tRecall: {:0.4f}'.format(recall_score(y_true, y_pred)))
    print('\tF1: {:0.4f}'.format(f1_score(y_true, y_pred)))
    print('\tAccuracy: {:0.4f}'.format(accuracy_score(y_true, y_pred)))

For file: 35score_abk_w2vec.xlsx
	Precision: 1.0000
	Recall: 0.1429
	F1: 0.2500
	Accuracy: 0.9231
For file: 38score_abk_w2vec.xlsx
	Precision: 0.4444
	Recall: 0.6667
	F1: 0.5333
	Accuracy: 0.9231
For file: 39score_abk_w2vec.xlsx
	Precision: 0.5000
	Recall: 0.0667
	F1: 0.1176
	Accuracy: 0.8333
For file: 45score_abk_w2vec.xlsx
	Precision: 0.7500
	Recall: 0.3750
	F1: 0.5000
	Accuracy: 0.9310
For file: 46score_abk_w2vec.xlsx
	Precision: 0.3333
	Recall: 0.2500
	F1: 0.2857
	Accuracy: 0.8438
For file: 62score_abk_w2vec.xlsx
	Precision: 0.0000
	Recall: 0.0000
	F1: 0.0000
	Accuracy: 0.9126
For file: 64score_abk_w2vec.xlsx
	Precision: 0.3750
	Recall: 0.2143
	F1: 0.2727
	Accuracy: 0.8431
For file: 65score_abk_w2vec.xlsx
	Precision: 1.0000
	Recall: 0.0741
	F1: 0.1379
	Accuracy: 0.7596
For file: 66score_abk_w2vec.xlsx
	Precision: 0.5556
	Recall: 0.2632
	F1: 0.3571
	Accuracy: 0.8022
For file: 88score_abk_w2vec.xlsx
	Precision: 0.4615
	Recall: 0.2222
	F1: 0.3000
	Accuracy: 0.6585
